In [83]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [33]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Loading Dataset**

In [120]:
df_x = pd.read_csv("world_owid.csv", header=0)
df_x.head()

,year,cement_co2,co2,coal_co2,flaring_co2,gas_co2,oil_co2
0,1750,NaN,9.351,9.351,NaN,NaN,NaN
1,1751,NaN,9.351,9.351,NaN,NaN,NaN
2,1752,NaN,9.354,9.354,NaN,NaN,NaN
3,1753,NaN,9.354,9.354,NaN,NaN,NaN
4,1754,NaN,9.358,9.358,NaN,NaN,NaN


# **Preprocessing Data**

In [121]:

df_x["year"] = pd.to_datetime(df_x["year"], format="%Y")
df_x.index.freq = 'A'

In [122]:
df_x.info()
df = df_x.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272 entries, 0 to 271
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   year         272 non-null    datetime64[ns]
 1   cement_co2   142 non-null    float64       
 2   co2          272 non-null    float64       
 3   coal_co2     272 non-null    float64       
 4   flaring_co2  72 non-null     float64       
 5   gas_co2      140 non-null    float64       
 6   oil_co2      167 non-null    float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 15.0 KB


In [123]:
test_dates = df_x["year"][130:]
print(test_dates.shape)
#print(df.head())

(142,)


In [124]:
df

,year,cement_co2,co2,coal_co2,flaring_co2,gas_co2,oil_co2
0,1750-01-01,NaN,9.351,9.351,NaN,NaN,NaN
1,1751-01-01,NaN,9.351,9.351,NaN,NaN,NaN
2,1752-01-01,NaN,9.354,9.354,NaN,NaN,NaN
3,1753-01-01,NaN,9.354,9.354,NaN,NaN,NaN
4,1754-01-01,NaN,9.358,9.358,NaN,NaN,NaN
...,...,...,...,...,...,...,...
267,2017-01-01,1507.923,36096.738,14506.974,391.992,7144.928,12242.628
268,2018-01-01,1569.218,36826.508,14746.831,412.116,7529.847,12266.017
269,2019-01-01,1617.507,37082.559,14725.978,439.254,7647.528,12345.653
270,2020-01-01,1637.537,35264.086,14174.564,407.584,7556.290,11191.809


Slicing of Dataframe for co2 and Coal Emissions and data from 1880 onwards

In [125]:
co2_df = df[['year','co2']].loc[(df['year'] >= '1880-01-01')]
coal_df = df[['year','coal_co2']].loc[(df['year'] >= '1880-01-01')]


# **1 Experimenting With Transforms to detrend the data. Log / Exponential Transforms**

In [126]:
# Apply log transformation to 'co2_emission' column
co2_df['Log_Transformed'] = np.log(co2_df['co2'])
coal_df['Log_Transformed'] = np.log(coal_df['coal_co2'])
# Drop NaN rows resulting from the log transformation
co2_df.dropna(inplace=True)
coal_df.dropna(inplace=True)

log_co2_df=co2_df.copy()
log_coal_df=coal_df.copy()

# **2 Differencing**

# After applying Log Transformation, we should differencing for making the data stationary, differencing was applyed 2-3 times depend on data, co2-coal

# log_coal_df

In [127]:
# Apply first-order differencing to the log-transformed values
co2_df['Log_Differenced_1'] = co2_df['Log_Transformed'].diff()
co2_df['Log_Differenced_2'] = co2_df['Log_Differenced_1'].diff()


In [128]:
# Apply first-order differencing to the log-transformed values
coal_df['Log_Differenced_1'] = coal_df['Log_Transformed'].diff()
coal_df['Log_Differenced_2'] = coal_df['Log_Differenced_1'].diff()

In [129]:
log_coal_df =coal_df.dropna().copy()
log_co2_df =co2_df.dropna().copy()

In [130]:
log_coal_df

,year,coal_co2,Log_Transformed,Log_Differenced_1,Log_Differenced_2
132,1882-01-01,912.817,6.816535,0.053619,0.022126
133,1883-01-01,974.149,6.881564,0.065029,0.011410
134,1884-01-01,982.679,6.890283,0.008718,-0.056311
135,1885-01-01,986.996,6.894666,0.004383,-0.004335
136,1886-01-01,995.975,6.903722,0.009056,0.004673
...,...,...,...,...,...
267,2017-01-01,14506.974,9.582385,0.010141,0.034186
268,2018-01-01,14746.831,9.598783,0.016399,0.006258
269,2019-01-01,14725.978,9.597368,-0.001415,-0.017814
270,2020-01-01,14174.564,9.559204,-0.038164,-0.036749


**Augmented Dicky Fuller Test**

In [131]:
from statsmodels.tsa.stattools import adfuller
X = log_coal_df[['year','Log_Differenced_1']].set_index("year").values
log_x = log_coal_df[['year','Log_Differenced_1']].set_index("year").values
result = adfuller(X)
log_result = adfuller(log_x)



print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))


print('\n\nADF Statistic: %f' % log_result[0])
print('p-value: %f' % log_result[1])
print('Critical Values:')
for key, value in log_result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistic: -4.013290
p-value: 0.001342
Critical Values:
	1%: -3.481
	5%: -2.884
	10%: -2.579


ADF Statistic: -4.013290
p-value: 0.001342
Critical Values:
	1%: -3.481
	5%: -2.884
	10%: -2.579


In [132]:
X = log_co2_df[['year','Log_Differenced_2']].set_index("year").values
log_x = log_co2_df[['year','Log_Differenced_2']].set_index("year").values
result = adfuller(X)
log_result = adfuller(log_x)



print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
    print('\t%s: %.3f' % (key, value))


print('\n\nADF Statistic: %f' % log_result[0])
print('p-value: %f' % log_result[1])
print('Critical Values:')
for key, value in log_result[4].items():
    print('\t%s: %.3f' % (key, value))

ADF Statistic: -7.305057
p-value: 0.000000
Critical Values:
	1%: -3.483
	5%: -2.884
	10%: -2.579


ADF Statistic: -7.305057
p-value: 0.000000
Critical Values:
	1%: -3.483
	5%: -2.884
	10%: -2.579


###### Augmented Dicky fuller test indicates that the data is stationary ADF - after **differencing**

##### Train-test Split

In [133]:
#coal_train_size = int(len(log_coal_df) * 0.9)
#coal_train, coal_test = [0:coal_train_size], X[coal_train_size:len(X)]


co2_data = log_co2_df[['year','Log_Differenced_2']].set_index("year").copy()
#co2_train_size = int(len(co2_data) -10)
co2_train_size = int(len(co2_data))
co2_train, co2_test = co2_data[0:co2_train_size], co2_data[11:co2_train_size]


In [134]:
co2_test

,Log_Differenced_2
year,
1893-01-01,-0.020275
1894-01-01,0.046323
1895-01-01,0.024247
1896-01-01,-0.026429
1897-01-01,0.014165
...,...
2017-01-01,0.016956
2018-01-01,0.004027
2019-01-01,-0.013087


In [136]:
co2_train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 140 entries, 1882-01-01 to 2021-01-01
Data columns (total 1 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Log_Differenced_2  140 non-null    float64
dtypes: float64(1)
memory usage: 2.2 KB


In [137]:
co2_train = pd.Series(co2_train['Log_Differenced_2'], index=pd.date_range(start='1882-01-01', periods=140, freq='AS-JAN'))
co2_test = pd.Series(co2_test['Log_Differenced_2'], index=pd.date_range(start='1893-01-01', periods=129, freq='AS-JAN'))

**Algorithm 1 - AR - Autoregressive Model**

In [138]:
from statsmodels.tsa.ar_model import AutoReg
from sklearn.metrics import mean_absolute_error, mean_squared_error

ar_model = AutoReg(co2_train, lags = 7)
ar_model = ar_model.fit()


In [139]:
#print(ar_model.summary())


In [140]:
#forecast_index = pd.date_range(start=f'{2021}-01-01', periods=10, freq='AS-JAN')

forecast1 = ar_model.predict(start=0, end=140 - 1, dynamic=False)
forecast1

1882-01-01         NaN
1883-01-01         NaN
1884-01-01         NaN
1885-01-01         NaN
1886-01-01         NaN
                ...   
2017-01-01    0.028126
2018-01-01   -0.002446
2019-01-01   -0.012658
2020-01-01   -0.000175
2021-01-01    0.053381
Freq: AS-JAN, Length: 140, dtype: float64

In [141]:
co2_df

,year,co2,Log_Transformed,Log_Differenced_1,Log_Differenced_2
130,1880-01-01,853.705,6.749586,NaN,NaN
131,1881-01-01,882.405,6.782651,0.033065,NaN
132,1882-01-01,931.922,6.837249,0.054598,0.021533
133,1883-01-01,991.032,6.898747,0.061498,0.006900
134,1884-01-01,1002.174,6.909927,0.011180,-0.050318
...,...,...,...,...,...
267,2017-01-01,36096.738,10.493958,0.015989,0.016956
268,2018-01-01,36826.508,10.513973,0.020015,0.004027
269,2019-01-01,37082.559,10.520902,0.006929,-0.013087
270,2020-01-01,35264.086,10.470620,-0.050282,-0.057211


In [142]:
co2_test

1893-01-01   -0.020275
1894-01-01    0.046323
1895-01-01    0.024247
1896-01-01   -0.026429
1897-01-01    0.014165
                ...   
2017-01-01    0.016956
2018-01-01    0.004027
2019-01-01   -0.013087
2020-01-01   -0.057211
2021-01-01    0.101676
Freq: AS-JAN, Name: Log_Differenced_2, Length: 129, dtype: float64

# **Reversing Differencing and Log Transformation on Co2**

# **Reverse Differencing and Log Transformation Function**

In [148]:
co2_df.tail(n=20)

,year,co2,Log_Transformed,Log_Differenced_1,Log_Differenced_2
252,2002-01-01,26281.037,10.176603,0.023601,0.015212
253,2003-01-01,27651.596,10.227439,0.050836,0.027235
254,2004-01-01,28636.695,10.262444,0.035005,-0.015830
255,2005-01-01,29614.602,10.296023,0.033579,-0.001427
256,2006-01-01,30593.117,10.328530,0.032507,-0.001071
257,2007-01-01,31506.789,10.357958,0.029428,-0.003080
258,2008-01-01,32085.836,10.376170,0.018212,-0.011216
259,2009-01-01,31564.031,10.359773,-0.016396,-0.034608
260,2010-01-01,33364.348,10.415243,0.055470,0.071866
261,2011-01-01,34487.012,10.448338,0.033095,-0.022375


In [154]:
# Reverse the first differencing and log transformation for the forecasted values
recovered_test_1 = co2_test.cumsum()
recovered_test_1 = recovered_test_1 + co2_df['Log_Differenced_1'].loc[(co2_df['year'] >= '1891-01-01')].iloc[1]  # Adding the last value of the training data to reverse first differencing

# Reverse the log transformation
recovered_test_2 = recovered_test_1.cumsum()
recovered_test_2 = recovered_test_2 + co2_df['Log_Transformed'].loc[(co2_df['year'] >= '1891-01-01')].iloc[1]   # Adding the last log-transformed value to reverse log transformation

# Recover original CO2 emission values
recovered_co2_test = np.exp(recovered_test_2)

# Create a DataFrame to hold the recovered test values
recovered_test_df = pd.DataFrame({
    'Recovered_CO2_test': recovered_co2_test
}, index=co2_test.index)  # Using the same index as the original 'co2_test' data

# Print the recovered forecasted values
print(recovered_test_df)

            Recovered_CO2_test
1893-01-01            1353.669
1894-01-01            1400.858
1895-01-01            1485.273
1896-01-01            1533.701
1897-01-01            1606.301
...                        ...
2017-01-01           36096.738
2018-01-01           36826.508
2019-01-01           37082.559
2020-01-01           35264.086
2021-01-01           37123.852

[129 rows x 1 columns]


In [152]:
# Reverse the first differencing and log transformation for the forecasted values
recovered_forecast_1 = forecast1.cumsum()
recovered_forecast_1 = recovered_forecast_1 + co2_df['Log_Differenced_1'].loc[(co2_df['year'] >= '1888-01-01')].iloc[1]  # Adding the last value of the training data to reverse first differencing

# Reverse the log transformation
recovered_forecast_2 = recovered_forecast_1.cumsum()
recovered_forecast_2 = recovered_forecast_2 + co2_df['Log_Transformed'].loc[(co2_df['year'] >= '1890-01-01')].iloc[1]   # Adding the last log-transformed value to reverse log transformation

# Recover original CO2 emission values
recovered_co2_forecast = np.exp(recovered_forecast_2)

# Create a DataFrame to hold the recovered forecast values
recovered_forecast_df = pd.DataFrame({
    'Recovered_CO2_Forecast': recovered_co2_forecast
}, index=co2_test.index)  # Using the same index as the original 'co2_test' data

# Print the recovered forecasted values
print(recovered_forecast_df)

            Recovered_CO2_Forecast
1893-01-01             1018.257932
1894-01-01             1022.415754
1895-01-01             1041.809901
1896-01-01             1015.895021
1897-01-01             1006.013706
...                            ...
2017-01-01                0.064241
2018-01-01                0.058626
2019-01-01                0.052829
2020-01-01                0.047597
2021-01-01                0.045235

[129 rows x 1 columns]


In [63]:
# Convert the array to a pandas DataFrame
#df_forecast = pd.DataFrame({'Forecasted_Values': forecast_original_scale})

In [64]:
recovered_forecast_df.iloc[7:]

,Recovered_CO2_Forecast
1889-01-01,859.840129
1890-01-01,873.488791
1891-01-01,840.365730
1892-01-01,802.304891
1893-01-01,781.640526
...,...
2017-01-01,3.125647
2018-01-01,2.949533
2019-01-01,2.748334
2020-01-01,2.560410


In [65]:
recovered_test_df.iloc[7:]

,Recovered_CO2_test
1889-01-01,1191.800
1890-01-01,1298.458
1891-01-01,1358.874
1892-01-01,1370.088
1893-01-01,1353.669
...,...
2017-01-01,36096.738
2018-01-01,36826.508
2019-01-01,37082.559
2020-01-01,35264.086


In [66]:
autoreg_mae = mean_absolute_error(recovered_forecast_df.iloc[7:],recovered_test_df.iloc[7:])
print(f"MAE : {autoreg_mae:3f}")

MAE : 12326.584896


In [73]:
results = pd.DataFrame({'Year': co2_test.index, 'Actual Emission': recovered_test_df['Recovered_CO2_test'], 'Predicted Emission': recovered_forecast_df['Recovered_CO2_Forecast']})

In [74]:
results

,Year,Actual Emission,Predicted Emission
1882-01-01,1882-01-01,931.922,NaN
1883-01-01,1883-01-01,991.032,NaN
1884-01-01,1884-01-01,1002.174,NaN
1885-01-01,1885-01-01,1009.671,NaN
1886-01-01,1886-01-01,1025.475,NaN
...,...,...,...
2017-01-01,2017-01-01,36096.738,3.125647
2018-01-01,2018-01-01,36826.508,2.949533
2019-01-01,2019-01-01,37082.559,2.748334
2020-01-01,2020-01-01,35264.086,2.560410


In [153]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100



In [72]:
mape_score = mean_absolute_percentage_error(recovered_forecast_df.iloc[120:],recovered_test_df.iloc[120:])
print(f"MAPE: {mape_score:.2f}%")

MAPE: 735213.13%


In [69]:
recovered_forecast_df

,Recovered_CO2_Forecast
1882-01-01,NaN
1883-01-01,NaN
1884-01-01,NaN
1885-01-01,NaN
1886-01-01,NaN
...,...
2017-01-01,3.125647
2018-01-01,2.949533
2019-01-01,2.748334
2020-01-01,2.560410


# **2 ARIMA**

# Normally you don't need to difference your target values because ARIMA support differencing (d), however I applied differencing ad Log Transform without changing for ARIMA since I already used them for AR - Autoregressive Model above.

# **Reverse Differencing and Log Transformation Function**

In [155]:
def reverseDiffer(forecast2, co2_df, co2_test):
  arima_forecast_1 = forecast2.cumsum()
  arima_forecast_1 = arima_forecast_1 + co2_df['Log_Differenced_1'].loc[(co2_df['year'] >= '1892-01-01')].iloc[1]  # Adding the last value of the training data to reverse first differencing

  # Reverse the log transformation
  arima_forecast_2 = arima_forecast_1.cumsum()
  arima_forecast_2 = arima_forecast_2 + co2_df['Log_Transformed'].loc[(co2_df['year'] >= '1892-01-01')].iloc[1]   # Adding the last log-transformed value to reverse log transformation

  # Recover original CO2 emission values
  arima_co2_forecast = np.exp(arima_forecast_2)

  # Create a DataFrame to hold the recovered forecast values
  arima_forecast_df = pd.DataFrame({'Recovered_CO2_Forecast_arima': arima_co2_forecast}, index=co2_test.index)  # Using the same index as the original 'co2_test' data
  return arima_forecast_df


In [156]:
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_squared_error
import numpy as np



# Define a list of possible values for p, d, and q
p_values = range(0, 20)  # You can adjust the range based on your requirements
d_values = [ 0,1,2,3,4,5]  # You can adjust the range based on your requirements
q_values = range(0, 15)  # You can adjust the range based on your requirements

mae_list = []
# starting point of mae, you amy change with a number that you don't expect as a result, and I just use starting point as 10000 which is really high MAE
best_mae = 10000 # starting point
best_order_ = []

# Try different combinations of p, d, and q
for p in p_values:
    for d in d_values:
        for q in q_values:


          try:
            # Fit ARIMA model with the current parameters
            model = ARIMA(co2_train, order=(p, d, q))
            model_fit = model.fit()

            # Make predictions on the validation set
            forecast_ = model_fit.forecast(steps=len(co2_test))

            # adjust in a reversing differencing function
            adj_arima_forecast=reverseDiffer(forecast_, co2_df, co2_test)

            # Calculate Mean Absolute Error (MAE) as the evaluation metric
            mae = mean_absolute_error(adj_arima_forecast, recovered_test_df)
            mae_list.append(mae)
            best_order_.append((p,d,q))
          except Exception as e:
            # Skip the combination if the model fitting fails
            continue
# find the index and value of minumum MAE and its parameters (p,d,q)
min_mae_index = mae_list.index(min(mae_list))
min_best_order =best_order_[min_mae_index]

# Fit the best ARIMA model with the selected order parameters
best_model = ARIMA(co2_train, order=(min_best_order[0], min_best_order[1], min_best_order[2]))
best_model_fit = best_model.fit()

# Print the summary of the best ARIMA model
print(best_model_fit.summary())

# Make predictions on the test set using the best ARIMA model
forecast_arima = best_model_fit.forecast(steps=10)

## adjust in a reversing differencing function
arima_forecast_df = reverseDiffer(forecast_arima, co2_df, co2_test)

Görüntülenen çıkış son 5000 satıra kısaltıldı.
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/usr/local/lib/python3.10/dist-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:402: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/statsmodels/tsa/statespace/sarimax.py:966: Us

ValueError: ignored

In [94]:
arima_mae = mean_absolute_error(arima_forecast_df,recovered_test_df)
print(f"MAE : {arima_mae:3f}")

MAE : 258.492133


In [95]:
mape_score = mean_absolute_percentage_error(arima_forecast_df,recovered_test_df)
print(f"MAPE: {mape_score:.2f}%")

MAPE: 0.71%


In [96]:
result_arima = pd.concat([recovered_test_df, arima_forecast_df['Recovered_CO2_Forecast_arima']], axis=1)
result_arima

,Recovered_CO2_test,Recovered_CO2_Forecast_arima
2012-01-01,35006.270,35205.033867
2013-01-01,35319.203,35284.524141
2014-01-01,35577.535,35562.816142
2015-01-01,35558.566,35592.686834
2016-01-01,35524.191,35509.657014
2017-01-01,36096.738,36248.029775
2018-01-01,36826.508,36662.533204
2019-01-01,37082.559,36671.655549
2020-01-01,35264.086,36798.971081
2021-01-01,37123.852,37096.802180
